In [ ]:
%load_ext autoreload
%autoreload 2

### Vector Store Initiation

In [4]:
# Collect documents from data-store:

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

# Load context files from directory:
loader = DirectoryLoader(
    path="../../data",
    glob="**/*.context"
)
data = loader.load()
print(f"Loaded {len(data)} documents.")

Loaded 50 documents.


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


# Split into chunks:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(data)

# Create vector store with embeddings:
vector_store = Chroma.from_documents(
    documents=all_splits,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
)

retriever = vector_store.as_retriever()

### Inititialize LLM Interface

In [13]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    base_url="http://172.17.0.1:7869",
    model="llama3.1:8b-instruct-q3_K_L",
    verbose=True,
)

### Build LangChaims

In [32]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.prompts import PromptTemplate
from operator import itemgetter


In [31]:
# Utilities functions for formatting output:
def format_context ( retrieved_contexts ):
    return "\n\n ------------".join(context.page_content for context in retrieved_contexts)


#### Simplest RAG Chain

In [24]:
qa_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context} 
    Answer:
    """
qa_prompt = PromptTemplate.from_template(qa_template)

# View the created Prompt:

rag_chain = (
    RunnableParallel(
        context = retriever, 
        question = RunnablePassthrough() ) \
    | qa_prompt
)

rag_chain.invoke( "What religion were the Normands?")

StringPromptValue(text='You are an assistant for question-answering tasks. \n    Use the following pieces of retrieved context to answer the question. \n    If you don\'t know the answer, just say that you don\'t know. \n    Use three sentences maximum and keep the answer concise.\n    Question: What religion were the Normands? \n    Context: [Document(metadata={\'source\': \'../../data/contexts/context_1.context\'}, page_content=\'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Fr

In [25]:
# Execute prompt with LLM to get response:
rag_chain = (
    RunnableParallel (
        context = retriever | format_context,
        question = RunnablePassthrough()
    ) \
    | qa_prompt
    | llm
)

rag_chain.invoke("What religion were the Normans?")

'The Normans were eventually Christians who became exponents of Catholic orthodoxy after assimilating into it. They adopted Christian piety as a part of their culture, which was distinct from their original Norse roots. This shift to Christianity is mentioned in the provided context.'

#### LLM Self Assessment

In [38]:
from enum import Enum
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class gradeEnum(str,Enum):
        correct = "correct"
        incorrect = "incorrect"
        
class LLMEvalResult(BaseModel):
    grade: gradeEnum = Field(description="Final grade label. Accepted labels : Correct, Incorrect")
    description: str = Field(description="Explanation of why the specific grade was assigned. Must be concise. Not more than 2 sentences")

json_parser = JsonOutputParser(pydantic_object=LLMEvalResult)

qa_eval_prompt_with_context_text = """
You are a teacher evaluating a test. 
You are provided with a question along with an answer for the question written by a student. Evaluate the question-answer pair using the provided context and provide feedback. Only mark the answer as correct if it agress with the provided context

{format_instructions}
Context : {context}
Question : {question}
Answer : {answer}
"""

qa_eval_prompt_with_context = PromptTemplate(
    template=qa_eval_prompt_with_context_text,
    input_variables=["question","answer","context"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

rag_chain = (
    RunnableParallel(
        context = retriever | format_context, 
        question = RunnablePassthrough() ) \
    | RunnableParallel(
        answer= qa_prompt | llm,
        question = itemgetter("question"),
        context = itemgetter("context")
    ) \
    | RunnableParallel(
        answer = itemgetter("answer"),
        question = itemgetter("question"),
        context = itemgetter("context"),
        evaluation = qa_eval_prompt_with_context | llm | json_parser
    )
)

rag_chain.invoke("What religion were the Normans?")

{'answer': 'The Normans were eventually exponents of Catholic orthodoxy after assimilating into Christian piety, but their initial culture and identity emerged from Norse raiders and pirates. They had no distinct pre-Christian religion mentioned in the context. Over time, they adopted Catholicism as a part of their new identity.',
 'question': 'What religion were the Normans?',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic id